In [6]:
%pip install -U langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 2.1 MB/s eta 0:00:00


In [3]:
from langchain.chat_models import init_chat_model
from google.colab import userdata

In [4]:
api_key =userdata.get('OPENAI_API_KEY')

In [7]:
llm = init_chat_model("gpt-4o-mini", model_provider="openai", api_key = api_key)

In [19]:
from langchain.tools import tool

@tool
def add(a: int, b: int) -> int:

  """Add a and b """
  return a + b

In [20]:
@tool
def subtract(a: int, b: int) -> int:

  """Subtract b from a """
  return a - b

In [21]:
@tool
def multiply(a: int, b: int) -> int:

  """Multiply a by b """
  return a * b

In [22]:
tool_map = {"add": add, "subtract": subtract, "multiply": multiply}


In [23]:
input = {"a": 1, "b": 2}
tool_map["add"].invoke(input)

3

In [25]:
tools = [add, subtract, multiply]
llm_with_tools = llm.bind_tools(tools)


In [26]:
tool_map["subtract"].invoke(input)

-1

In [28]:
from langchain_core.messages import HumanMessage

query = "What is 3 + 2?"
chat_history = [HumanMessage(content=query)]

In [49]:
print("Chat history before invoke:", chat_history)
response_1 = llm_with_tools.invoke(chat_history)
chat_history.append(response_1)

print(type(response_1))
#print(response_1)

Chat history before invoke: [HumanMessage(content='What is 3 + 2?', additional_kwargs={}, response_metadata={})]
<class 'langchain_core.messages.ai.AIMessage'>


In [50]:
tool_calls_1 = response_1.tool_calls

tool_1_name = tool_calls_1[0]["name"]
tool_1_args = tool_calls_1[0]["args"]
tool_call_1_id = tool_calls_1[0]["id"]

print(f'tool name:\n{tool_1_name}')
print(f'tool args:\n{tool_1_args}')
print(f'tool call ID:\n{tool_call_1_id}')

tool name:
add
tool args:
{'a': 3, 'b': 2}
tool call ID:
call_Xk1yYFpdUcYjf9rFLVhaBch8


In [51]:
from langchain_core.messages import ToolMessage

tool_response = tool_map[tool_1_name].invoke(tool_1_args)
tool_message = ToolMessage(content=tool_response, tool_call_id=tool_call_1_id)

print(tool_message)

content='5' tool_call_id='call_Xk1yYFpdUcYjf9rFLVhaBch8'


In [52]:
chat_history.append(tool_message)

In [53]:
answer = llm_with_tools.invoke(chat_history)
print(type(answer))
print(answer.content)

<class 'langchain_core.messages.ai.AIMessage'>
3 + 2 equals 5.


In [56]:
class ToolCallingAgent:
    def __init__(self, llm):
        self.llm_with_tools = llm.bind_tools(tools)
        self.tool_map = tool_map

    def run(self, query: str) -> str:
        # Step 1: Initial user message
        chat_history = [HumanMessage(content=query)]

        # Step 2: LLM chooses tool
        response = self.llm_with_tools.invoke(chat_history)
        if not response.tool_calls:
            return response.content # Direct response, no tool needed
        # Step 3: Handle first tool call
        tool_call = response.tool_calls[0]
        tool_name = tool_call["name"]
        tool_args = tool_call["args"]
        tool_call_id = tool_call["id"]

        # Step 4: Call tool manually
        tool_result = self.tool_map[tool_name].invoke(tool_args)

        # Step 5: Send result back to LLM
        tool_message = ToolMessage(content=str(tool_result), tool_call_id=tool_call_id)
        chat_history.extend([response, tool_message])

        # Step 6: Final LLM result
        final_response = self.llm_with_tools.invoke(chat_history)
        return final_response.content

In [57]:
my_agent = ToolCallingAgent(llm)

print(my_agent.run("one plus 2"))

print(my_agent.run("one - 2"))

print(my_agent.run("three times two"))

One plus two equals three.
The result of \( 1 - 2 \) is \( -1 \).
Three times two is 6.
